In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir, rename
from os.path import join, basename, isdir, isfile

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from os.path import join

In [ ]:
# read_csv = False

In [ ]:
# if read_csv:
#     doses_aplicadas = pd.concat([pd.read_csv(join('base',f"{x}.csv"), sep=';', low_memory=False, dtype=str) for x in ['P1','P2','P3']], ignore_index=True)
#     doses_aplicadas.to_pickle('open_data_sus.pkl',compression='bz2')
# else:
#     doses_aplicadas = pd.read_pickle('open_data_sus.pkl',compression='bz2')

In [ ]:
from bulletin.systems import eSUS
esus = eSUS()
esus.load('esus',compress=False)

In [ ]:
doses_aplicadas = esus.df
doses_aplicadas.dtypes

In [ ]:
doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'].isin(['Dose','Única']),'vacina_descricao_dose'] = 'Única'

In [ ]:
doses_aplicadas['paciente_idade'] = doses_aplicadas['paciente_idade'].fillna(-1).astype(int)

In [ ]:
doses_aplicadas['paciente_enumSexoBiologico'].unique()

In [ ]:
try:
    del(doses_aplicadas['faixa_etaria'])
except:
    pass

doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 5) & (doses_aplicadas['paciente_idade'] < 12), 'faixa_etaria'] = '5-11 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 12) & (doses_aplicadas['paciente_idade'] < 18), 'faixa_etaria'] = '12-17 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 18) & (doses_aplicadas['paciente_idade'] <= 19), 'faixa_etaria'] = '18-19 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 20) & (doses_aplicadas['paciente_idade'] <= 24), 'faixa_etaria'] = '20-24 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 25) & (doses_aplicadas['paciente_idade'] <= 29), 'faixa_etaria'] = '25-29 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 30) & (doses_aplicadas['paciente_idade'] <= 34), 'faixa_etaria'] = '30-34 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 35) & (doses_aplicadas['paciente_idade'] <= 39), 'faixa_etaria'] = '35-39 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 40) & (doses_aplicadas['paciente_idade'] <= 44), 'faixa_etaria'] = '40-44 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 45) & (doses_aplicadas['paciente_idade'] <= 49), 'faixa_etaria'] = '45-49 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 50) & (doses_aplicadas['paciente_idade'] <= 54), 'faixa_etaria'] = '50-54anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 55) & (doses_aplicadas['paciente_idade'] <= 59), 'faixa_etaria'] = '55-59 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 60) & (doses_aplicadas['paciente_idade'] <= 64), 'faixa_etaria'] = '60-64 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 65) & (doses_aplicadas['paciente_idade'] <= 69), 'faixa_etaria'] = '65-69 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 70) & (doses_aplicadas['paciente_idade'] <= 74), 'faixa_etaria'] = '70-74 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 75) & (doses_aplicadas['paciente_idade'] <= 79), 'faixa_etaria'] = '75-79 anos'
doses_aplicadas.loc[(doses_aplicadas['paciente_idade'] >= 80), 'faixa_etaria'] = '>80 anos'



# Relatório Geral de Doses Aplicadas, segundo a Dose e Faixa Etária

In [ ]:
RelatorioGEral = doses_aplicadas.groupby(['estabelecimento_municipio_nome', 'faixa_etaria', 'vacina_descricao_dose'])[['document_id']].count().unstack(['faixa_etaria','vacina_descricao_dose']).sort_index(level='faixa_etaria', axis=1)
RelatorioGEral

# Relatório Acácia

In [ ]:
DosesGeral = doses_aplicadas.groupby(['faixa_etaria', 'vacina_descricao_dose'])[['document_id']].count().unstack(['vacina_descricao_dose']).sort_index(level='faixa_etaria', axis=0)
DosesGeral

In [ ]:
DosesGeral.to_excel('DosesGeral.xlsx')

# Relatório Esquema Iniciado contra Covid-19

In [ ]:
EsquemaIniciado = doses_aplicadas.loc[(doses_aplicadas['vacina_descricao_dose'] == "1ª Dose") | (doses_aplicadas['vacina_descricao_dose'] == "Única") ]  

DosesporFaixa_Idade = EsquemaIniciado.groupby(['estabelecimento_municipio_nome', 'faixa_etaria'])[['document_id']].count().unstack(['faixa_etaria']).sort_index(level='faixa_etaria', axis=1)
DosesporFaixa_Idade

In [ ]:
DosesporFaixa_Idade.to_excel("DosesporFaixa_Idade.xlsx")

# Relatório de Primeiras Doses Aplicadas contra Covid-19

In [ ]:
PrimeirasDoses = doses_aplicadas.loc[(doses_aplicadas['vacina_descricao_dose'] == "1ª Dose")]  

PrimeirasDoses = PrimeirasDoses.groupby(['estabelecimento_municipio_nome', 'faixa_etaria'])[['document_id']].count().unstack(['faixa_etaria']).sort_index(level='faixa_etaria', axis=1)
PrimeirasDoses

In [ ]:
PrimeirasDoses.to_excel("PrimeirasDoses.xlsx")

# Relatório de Segundas Doses Aplicadas contra Covid-19

In [ ]:
SegundaDose = doses_aplicadas.loc[(doses_aplicadas['vacina_descricao_dose'] == "2ª Dose")]  

SegundaDose_faixa = SegundaDose.groupby(['estabelecimento_municipio_nome', 'faixa_etaria'])[['document_id']].count().unstack(['faixa_etaria']).sort_index(level='faixa_etaria', axis=1)
SegundaDose_faixa

In [ ]:
SegundaDose_faixa.to_excel("SegundaDose_Faixa.xlsx")

# Relatório de Segundas Doses e Únicas

In [ ]:
SegundaDoseeUnicas = doses_aplicadas.loc[(doses_aplicadas['vacina_descricao_dose'] == "2ª Dose") | (doses_aplicadas['vacina_descricao_dose'] == "Única") ]  

SegundaDoseeUnicas = SegundaDoseeUnicas.groupby(['estabelecimento_municipio_nome', 'faixa_etaria'])[['document_id']].count().unstack(['faixa_etaria']).sort_index(level='faixa_etaria', axis=1)
SegundaDoseeUnicas

In [ ]:
SegundaDoseeUnicas.to_excel("SegundaDoseeUnicas.xlsx")

# Consolidado de Doses segundo a Descrição de Dose

In [ ]:
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "1ª Dose Redoses_aplicadasacinação"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "2ª Dose Redoses_aplicadasacinação"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "1º Reforço"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "2º Reforço"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "3ª Dose"]
# doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "Reforço"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['doses_aplicadasacina_descricao_dose'] != "Redoses_aplicadasacinação"]
# doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['vacina_descricao_dose'] != "Dose Adicional"]
# doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['doses_aplicadasacina_descricao_dose'] != "1ª Dose"]
# doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['doses_aplicadasacina_descricao_dose'] != "Dose"]
# doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['doses_aplicadasacina_descricao_dose'] != "Única"]
doses_aplicadas = doses_aplicadas.loc[doses_aplicadas['paciente_enumSexoBiologico'] != "I"]


doses_aplicadas.loc[doses_aplicadas['paciente_enumSexoBiologico'].isna(), "paciente_enumSexoBiologico"] = "I"

In [ ]:
ConsolidadoDoses = doses_aplicadas.groupby(['vacina_descricao_dose'])[['document_id']].count()
ConsolidadoDoses

In [ ]:
ConsolidadoDoses.to_excel("Consolidado de Doses Aplicadas.xlsx")

# Relatório de Chamada por Dose

In [ ]:
ChamadaDOSE = doses_aplicadas.loc[(doses_aplicadas['faixa_etaria']=='5-11 anos') | (doses_aplicadas['faixa_etaria']=='12-17 anos')].groupby(['estabelecimento_municipio_nome', 'faixa_etaria'])[['document_id']].count().unstack(['faixa_etaria'])
ChamadaDOSE

# Valores a Testar

In [ ]:
arquivoInt = doses_aplicadas.loc[doses_aplicadas['sistema_origem']=='Revacinação']
arquivoInt